<a href="https://colab.research.google.com/gist/Eldar1999/1448e24157a6898f96b14b6457d2c101/untitled3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 1

In [8]:
import cv2
import numpy as np
import os
import glob

from google.colab.patches import cv2_imshow

1. Определение размеров шахматной доски
2. Определение критериев для завершения поиска углов доски





In [9]:

CHECKERBOARD = (10,14)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

3. Создание вектора для хранения векторов трехмерных точек для каждого изображения шахматной доски
4. Создание вектора для хранения векторов 2D точек для каждого изображения шахматной доски

In [10]:
objpoints = []
imgpoints = []



5. Определение мировых координат для 3D точек


In [11]:
objp = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
prev_img_shape = None

6. Извлечение пути отдельного изображения, хранящегося в данном каталоге

In [12]:
images = glob.glob('./*.JPG')

In [ ]:
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Найти углы шахматной доски
    # Если на изображении найдено нужное количество углов, тогда ret = true
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
    
    """
    Если желаемый номер угла обнаружен,
    уточняем координаты пикселей и отображаем
    их на изображениях шахматной доски
    """
    if ret == True:
        objpoints.append(objp)
        # уточнение координат пикселей для заданных 2d точек.
        corners2 = cv2.cornerSubPix(gray, corners, (11,11),(-1,-1), criteria)
      
        imgpoints.append(corners2)

        # Нарисовать и отобразить углы
        img = cv2.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)

In [ ]:
h,w = img.shape[:2]

"""
Выполнение калибровки камеры с помощью
Передача значения известных трехмерных точек (объектов)
и соответствующие пиксельные координаты
обнаруженные углы (imgpoints)
"""
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

print("Camera matrix : ")
print(mtx)
print("dist : ")
print(dist)
print("rvecs : ")
print(rvecs)
print("tvecs : ")
print(tvecs)